### 1. Import required modules

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from datetime import datetime
from rendseq import file_funcs
from rendseq import utility_funcs
from rendseq import ramps_steps as rmp

### 2. User-defined inputs

* Required:
    + Input appropriate filename for the raw reads .wig file
    + Input appropriate filename for the precomputed peaks .wig file
    +  Input desired paramaters: window_width, blur_width, min_cutoff
        + window_width = 400, blur_width = 200 worked well for B. subtilis ramps, which were mostly long (1000+ bps)
* Optional:
    + Change sample_name, which will impact the output filenames
    + Customize output filenames if desired

In [ ]:
## input filenames
reads_filename = 'Example_Data/example_wig_Jean_5f_subset.wig'
peaks_filename = 'Example_Data/example_thresh_peaks.wig'

## parameters
window_width = 400
blur_width = 200
min_cutoff = 700 ## min_cutoff > window_width + blur_width

## output filename details
sample_name = "Jean_5f"
path = ""
inc_output_filename = "{p}{n}_inc_{w}_{b}_min{c}.csv".format(p = path, n = sample_name, w = window_width, b = blur_width, c = min_cutoff)
dec_output_filename = "{p}{n}_dec_{w}_{b}_min{c}.csv".format(p = path, n = sample_name, w = window_width, b = blur_width, c = min_cutoff)
wig_output_filename = "{p}{n}_{w}_{b}_min{c}.wig".format(p = path, n = sample_name, w = window_width, b = blur_width, c = min_cutoff)

## prints the output filenames
print('.csv output filename for increasing stretches: ', inc_output_filename)
print('.csv output filename for decreasing stretches: ', dec_output_filename)
print('.wig output filename (both increasing and decreasing): ', wig_output_filename)

### 3. Process the data

#### 3.1 Pre-processing

Read in the data, shave peaks, zero-pad the reads

In [ ]:
## read in the reads data and zscores
reads_raw, chrom = file_funcs.open_wig(reads_filename)

## find the peaks and shave them
peaks, chrom = file_funcs.open_wig(peaks_filename)
reads_smoothed = utility_funcs.smooth_peaks(peaks, reads_raw)

## zero-pad the reads
reads_pad = utility_funcs.zero_padding(reads_smoothed)
reads = utility_funcs.reads_pad.copy()

#### 3.2 Slope blurring

Creates a ReadsData object and calculates cumulative sum, blurred data, and derivative of blurred data.

In [ ]:
reads_obj = rmp.ReadsData(reads)
reads_obj.process_data(window_width = window_width, blur_width = blur_width)

#### 3.3 Finding positive and negative stretches

Creates two StretchData objects, one for increasing stretches and one for decreasing stretches

In [ ]:
## increasing stretches
inc = rmp.StretchData(reads_obj, 'positive')    # create object
inc.stretch_search()                            # find all increasing stretches
inc.filter_stretch_length(min_gap = min_cutoff) # apply a minimum cutoff for stretch length

## decreasing stretches
dec = StretchData(reads_obj, 'negative')
dec.stretch_search()
dec.filter_stretch_length(min_gap = min_cutoff)

## print out summary
print('-- Increasing stretches --',
      '\nbefore length filtering: ', inc.search_starts.shape[0], 
      '\nafter length filtering: ', inc.starts.shape[0])

print('\n-- Decreasing stretches --',
      '\nbefore length filtering: ', dec.search_starts.shape[0], 
      '\nafter length filtering: ', dec.starts.shape[0])

### 4. Output the results to .csv and .wig

Expect 3 files:
* Increasing stretches .csv
* Decreasing stretches .csv
* Combined .wig with both increasing and decreasing stretches

In [ ]:
inc.output_csv(inc_output_filename)
dec.output_csv(dec_output_filename)
rmp.output_to_wig(inc, dec, wig_output_filename, chrom)